In [37]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [38]:
llm_config = {
    "model": "gpt-4-turbo",
    "api_key": os.getenv("OPENAI_API_KEY"),
}

In [39]:
import autogen

In [40]:
user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message="Give the task, and send "
    "instructions to writer to refine the blog post.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

In [41]:
planner = autogen.ConversableAgent(
    name="Planner",
    system_message="Given a task, please determine "
    "what information is needed to complete the task. "
    "Please note that the information will all be retrieved using"
    " Python code. Please only suggest information that can be "
    "retrieved using Python code. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps. If a step fails, try to "
    "workaround",
    description="Planner. Given a task, determine what "
    "information is needed to complete the task. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps",
    llm_config=llm_config,
)

In [42]:
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    description="Write code based on the plan " "provided by the planner.",
)

In [43]:
executor = autogen.ConversableAgent(
    name="Executor",
    description="Execute the code written by the " "engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "group_chat",
        "use_docker": False,
    },
)

In [44]:
writer = autogen.ConversableAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="Writer. "
    "Please write blogs in markdown format (with relevant titles)"
    " and put the content in pseudo ```md``` code block. "
    "You take feedback from the admin and refine your blog.",
    description="After all the info is available, "
    "write blogs based on the code execution results and take "
    "feedback from the admin to refine the blog. ",
)

In [45]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10,
    allowed_or_disallowed_speaker_transitions={
        user_proxy: [engineer, writer, executor, planner],
        engineer: [user_proxy, executor],
        writer: [user_proxy, planner],
        executor: [user_proxy, engineer, planner],
        planner: [user_proxy, engineer, writer],
    },
    speaker_transitions_type="allowed",
)

In [46]:
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

groupchat_result = user_proxy.initiate_chat(
    manager,
    message="Write a blog post about the stock proce performace of Nvidia in the past month. Today is 2025-01-27",
)

Admin (to chat_manager):

Write a blog post about the stock proce performace of Nvidia in the past month. Today is 2025-01-27

--------------------------------------------------------------------------------

Next speaker: Planner


>>>>>>>> USING AUTO REPLY...
Planner (to chat_manager):

To write a blog post about Nvidia's stock price performance over the past month with Python code, we'll follow these steps:

1. **Retrieve Historical Stock Data**: We'll gather Nvidia's historical stock price data for the past month.
2. **Analyze the Data**: Calculate performance metrics like the opening, closing, highest, and lowest prices, as well as percentage changes.
3. **Visualize the Data**: Generate graphs to visually represent the stock price movement over the past month.
4. **Write the Blog Post**: Based on the analysis, write the content of the blog post discussing Nvidia's stock performance.

### Step 1: Retrieve Historical Stock Data
To begin, you need the stock price data for Nvidia (tic